# Using regular expressions to help with Wordle

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/rwcitek/MyBinder.demo/main?labpath=%2FRegular.Expressions%2Fwordle.python.ipynb)

### Background

Wordle is a word-guessing game where the program picks a five letter word from its list and you have six tries to guess it.  
That sounds like a pretty big search space for guessing.  I wonder just how big that search space is and if it can be made smaller with the hints that Wordle gives you about previous guesses.

### Task 1: How big is the search space?

In [1]:
!ls -la /usr/share/dict/words

lrwxrwxrwx 1 root root 30 Oct 17 22:21 /usr/share/dict/words -> /etc/dictionaries-common/words


In [2]:
!readlink -f /usr/share/dict/words

/usr/share/dict/american-english-insane


The "insanely long list of american words" is installed.
I wonder how many words are in it.

In [3]:
words = !cat /usr/share/dict/words
len(words)

654749

In [4]:
!head /usr/share/dict/words

A
A'asia
A's
AATech
AATech's
AAeE
AAeE's
AAgr
AAgr's
AAvTech


About 650k words.
Of those, I wonder how many are five-letter words.
To answer that, we'll load Python's regular expression library ( re ) and numpy.

In [3]:
import re
import numpy as np


In [6]:
regex = r'^[a-z]{5}$'
words_5 = [ i for i in words if re.search(regex , i) ]
with open("/tmp/words.5-letters.txt", "w") as file:
    for word in words_5:
        file.write(f"{word}\n")


In [7]:
!head /tmp/words.5-letters.txt
!wc /tmp/words.5-letters.txt


aahed
aalii
aargh
aaron
abaca
abaci
aback
abacs
abada
abaff
 17318  17318 103908 /tmp/words.5-letters.txt


About 17k.  That's a lot smaller, but still a pretty big search space.

### Task 2: What's a good initial guess to reduce the search space?

I wonder what letters appear most often in that list of five-letter words.
Let's create a frequency list.

In [9]:
letters = list("".join(words_5))
sorted(np.column_stack((np.unique(letters, return_counts=True))), key=lambda x: int(x[1]), reverse=True)[:10]


[array(['a', '8789'], dtype='<U21'),
 array(['e', '8609'], dtype='<U21'),
 array(['s', '7742'], dtype='<U21'),
 array(['o', '5770'], dtype='<U21'),
 array(['r', '5505'], dtype='<U21'),
 array(['i', '5260'], dtype='<U21'),
 array(['l', '4544'], dtype='<U21'),
 array(['t', '4518'], dtype='<U21'),
 array(['n', '4284'], dtype='<U21'),
 array(['u', '3598'], dtype='<U21')]

Let's see if there is a five-letter word that contains the top five letters in that frequency list.

In [10]:
[ i for i in words_5 
    if re.search(r'a', i) and
       re.search(r'e', i) and
       re.search(r's', i) and
       re.search(r'o', i) and
       re.search(r'r', i)
]


['arose', 'seora', 'soare']

Woohoo! That worked.

### Task 3: Evaluate the guess, i.e. play Wordle

There are a number of Wordle sites, including the [New York Times](https://www.nytimes.com/games/wordle/index.html).  I'm using the one from wordleplay.com because I can point to a specific game, for example, this one:

https://wordleplay.com?challenge=bGVhZHk=


#### The hints

The goal to playing Wordle is to guess a five letter word that the system has picked from a pool of possible five-letter words.  Once you make an initial guess using a valid word ( a non-word guess is not allowed ), then Wordle provides three hints for each letter in the word:

1. a match of both the letter and its position in the words
2. a mismatch, i.e. the letter is not anywhere in the word
3. a match of letter but not position

This last hint is actually two hints in one.  So there are actually four hints.  More on that later.


If the first word guess is "arose", then the results are as follows, using a two-character combination of letter and hint to encode the results:

1. A3
1. R2
1. O2
1. S2
1. E3

This means the letters R, O, and S are not in the word ( hint 3 ).  And the letters A and E are in the word, but in the wrong position ( hint 2 ).  Let's generate a regular expression for letters that obey rule two and use Python's regular expressions to filter words from the list of five-letter words.


#### Regular Expression for Second Hint

In [11]:
words_1_2 = [ i for i in words_5
             if not re.search(r'r', i) and 
                not re.search(r'o', i) and 
                not re.search(r's', i)
]
len(words_1_2)

4656

However, we can shorten that by using a character class:

In [12]:
words_1_2 = [ i for i in words_5 if not re.search(r'[ros]', i) ]
len(words_1_2)


4656

We've gone from 17k down to about 5k with just one hint.  Now to tackle hint 3. 

#### Regular Expression for Third Hint - Part 1

The third hint tells us that the letter 'a' is somewhere in the word just not at the first position.  We can use a regular expressing with an anchor to filter those away.

In [13]:
len( [ i for i in words_1_2 if not re.search(r'^a', i) ] )


4222

The caret ( '^' ) is a symbol that matches the beginning of a string.  And since the list is one word per string, it matches the beginning of a word.  As such it is known as an anchor because it anchors the regular expression to a fixed location, the beginning of the string.  So this regular expression matches all words that begin with the letter 'a'.  This allows words like 'babby' to pass through.  But words like 'aback' to be filtered out.

In [14]:
[ i for i in ["babby","aback"] if not re.search(r'^a', i) ]


['babby']

We can do something similar with the letter 'e', but with a twist.

In [15]:
len( [ i for i in words_1_2 if not re.search(r'e$', i) ] )


3918

In this case, we anchored the regular expression with a dollar symbol ( '$' ), which matches the end of the line.  So the regular expression matches all words that end with an 'e'.

We can combine those two commands into a single pipeline.

In [16]:
len( [ i for i in words_1_2 if not re.search(r'^a', i) and
                               not re.search(r'e$', i)
] )


3573

#### Regular Expression for Third Hint - Part 2

But the third hit tells us something more than only what letters are not in the list at a position.  It also tells us what *is* in the word.  In this case, both the letters 'a' and 'e' are in the word, just not at the position that we guessed.  We can therefore filter for words that contain both letters.

In [17]:
word_1_3 = [ i for i in words_1_2 if not re.search(r'^a', i) and
                                     not re.search(r'e$', i) and
                                         re.search(r'a', i) and
                                         re.search(r'e', i)
]
len(word_1_3)

526

With just one guess and two hints we have gone from a search space of about 17k down to 526 with the help of regular expressions.  Now, the question is, what should be our next guess?

### Task 4: Guess second word

At this point we redo some of the commands that we've done initially: generate a frequency table of letters and find a word that has the most common letters.

In [18]:
letters = list("".join(word_1_3))
sorted(np.column_stack((np.unique(letters, return_counts=True))), key=lambda x: int(x[1]), reverse=True)[:10]


[array(['e', '542'], dtype='<U21'),
 array(['a', '541'], dtype='<U21'),
 array(['l', '206'], dtype='<U21'),
 array(['d', '180'], dtype='<U21'),
 array(['t', '158'], dtype='<U21'),
 array(['n', '152'], dtype='<U21'),
 array(['m', '91'], dtype='<U21'),
 array(['c', '85'], dtype='<U21'),
 array(['h', '82'], dtype='<U21'),
 array(['b', '81'], dtype='<U21')]

Since 'a' and 'e' are already part of our inclusion search, we can skip searching for those.

In [19]:
len( [ i for i in word_1_3 if re.search(r'l', i) and
                              re.search(r'd', i) and
                              re.search(r't', i)
] )


5

In [20]:
[ i for i in word_1_3 if re.search(r'l', i) and
                         re.search(r'd', i) and
                         re.search(r't', i)
]


['dalet', 'dealt', 'delta', 'lated', 'taled']

Using 'delta' as our next guess, we get these hints:
1. D3
1. E1
1. L3
1. T2
1. A3

#### Regular Expression for First Hint

We now have a match for one letter, a letter 'e' at position 2.

In [21]:
word_2_1 = [ i for i in word_1_3 if re.search(r'^.e', i) ]
len(word_2_1)

211

#### Regular Expression for Second Hint

In [22]:
word_2_2 = [ i for i in word_2_1 if not re.search(r't', i) ]
len(word_2_2)

148

#### Regular Expression for Third Hint - Part 1

In [23]:
len(
[ i for i in word_2_2 if not re.search(r'^d', i) and
                         not re.search(r'^..l', i) and
                         not re.search(r'^....a', i)
]
)

80

#### Regular Expression for Third Hint - Part 2

In [24]:
word_2_3 = [ i for i in word_2_2 if not re.search(r'^d', i) and
                                    not re.search(r'^..l', i) and
                                    not re.search(r'^....a', i) and
                                        re.search(r'd', i) and
                                        re.search(r'l', i)
    ]
len(word_2_3)

6

### Task 5: Guess third word

The search space is down to six words that each contain the four known letters.

In [25]:
letters = list("".join(word_2_3))
sorted(np.column_stack((np.unique(letters, return_counts=True))), key=lambda x: int(x[1]), reverse=True)[:10]


[array(['a', '6'], dtype='<U21'),
 array(['d', '6'], dtype='<U21'),
 array(['e', '6'], dtype='<U21'),
 array(['l', '6'], dtype='<U21'),
 array(['h', '1'], dtype='<U21'),
 array(['m', '1'], dtype='<U21'),
 array(['n', '1'], dtype='<U21'),
 array(['p', '1'], dtype='<U21'),
 array(['w', '1'], dtype='<U21'),
 array(['y', '1'], dtype='<U21')]

At this point, there are so few words that we can generate a list of all substrings of the words and look at their frequencies.

In [26]:
from itertools import combinations
substrings = [ [ word[x:y] for x, y in combinations(range(len(word)+1), r=2)]
                             for word in word_2_3 ]
sorted(np.column_stack((np.unique(substrings, return_counts=True))), key=lambda x: int(x[1]), reverse=True)[:10]


[array(['a', '6'], dtype='<U21'),
 array(['d', '6'], dtype='<U21'),
 array(['e', '6'], dtype='<U21'),
 array(['l', '6'], dtype='<U21'),
 array(['al', '4'], dtype='<U21'),
 array(['ea', '3'], dtype='<U21'),
 array(['ad', '2'], dtype='<U21'),
 array(['ald', '2'], dtype='<U21'),
 array(['da', '2'], dtype='<U21'),
 array(['dal', '2'], dtype='<U21')]

In addition to the substrings, we can note their position in the word and calculate their frequencies.

In [27]:
from itertools import combinations
pos_substrings = [ [ f"{x}:{word[x:y+1]}" for x, y in combinations(range(len(word)), r=2)]
                             for word in word_2_3 ]
sorted(np.column_stack((np.unique(pos_substrings, return_counts=True))), key=lambda x: int(x[1]), reverse=True)[:20]


[array(['1:ea', '3'], dtype='<U21'),
 array(['0:le', '2'], dtype='<U21'),
 array(['1:eal', '2'], dtype='<U21'),
 array(['1:eald', '2'], dtype='<U21'),
 array(['1:ed', '2'], dtype='<U21'),
 array(['1:eda', '2'], dtype='<U21'),
 array(['1:edal', '2'], dtype='<U21'),
 array(['2:al', '2'], dtype='<U21'),
 array(['2:ald', '2'], dtype='<U21'),
 array(['2:da', '2'], dtype='<U21'),
 array(['2:dal', '2'], dtype='<U21'),
 array(['3:al', '2'], dtype='<U21'),
 array(['3:ld', '2'], dtype='<U21'),
 array(['0:he', '1'], dtype='<U21'),
 array(['0:hea', '1'], dtype='<U21'),
 array(['0:heal', '1'], dtype='<U21'),
 array(['0:heald', '1'], dtype='<U21'),
 array(['0:lea', '1'], dtype='<U21'),
 array(['0:lead', '1'], dtype='<U21'),
 array(['0:leady', '1'], dtype='<U21')]

The top substring, 'ae', can be combined with the next highest non-overlapping substrings, 'al' or 'ld', so we'll search for words that contain one of those.

In [28]:
[ i for i in word_2_3 if re.search(r'ea', i) and
                         re.search(r'ld', i)
]


['heald', 'weald']

Let's try 'heald' and the result is:

1. H2
1. E1
1. A1
1. L3
1. D3


#### Regular Expression for First Hint

In [29]:
word_3_1 = [ i for i in word_2_3 if re.search(r'^..a', i) ]
word_3_1

['heald', 'leady', 'weald']

#### Regular Expression for Second Hint

In [30]:
word_3_2 = [ i for i in word_3_1 if not re.search(r'h', i) ]
word_3_2

['leady', 'weald']

#### Regular Expression for Third Hint - Part 1

We can skip this step because we already know that 'l' and 'd' are in the word, we just don't know where.  However ....

#### Regular Expression for Third Hint - Part 2


... we do know that 'l' and 'd' are not at the end of the word

In [31]:
word_3_3 = [ i for i in word_3_2 if not re.search(r'^...l', i) and
                                    not re.search(r'^....d', i)
]
word_3_3

['leady']

And that leaves just one word ... 'leady'.

Let's try it ...


# Oooooo! That's a Bingo!

[Is that the way you say it?](https://youtu.be/Ugpg8XruhVk)


<iframe width="560" height="315" src="https://www.youtube.com/embed/Ugpg8XruhVk" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

